In [0]:
import pandas as pd
import math

Ulaz:
1.   Tabela sa kolonama: Apoteka, Stanje, Izlaz.
2.   Kolicnik koji definise visak.



In [0]:
tabela=pd.read_csv("https://raw.githubusercontent.com/vicasa/Moma/master/Sortis30x20.csv?token=AKSLGDMAS3FXKBSU2GWEH226SRP3A")
defVishka=2.0

Pravim kolonu trenutnih kolicnika.

In [0]:
tabela["Kolicnik"]=tabela["Stanje"]/tabela["Izlaz"]

Racunam teorijski najbolji kolicnik za sve Apoteke.
Ne koristim ga, ali cisto da ga vidim.

In [4]:
k_teorija=sum(tabela["Stanje"])/sum(tabela["Izlaz"])
k_teorija

1.29658410089356

Broj apoteka u "igri".

In [0]:
n=tabela.shape[0]

Odredjujem vishkove i pravim kolonu sa vishkovima.

In [0]:
k_vishkovi=[max([i-defVishka,0]) for i in tabela['Kolicnik']]
vishkovi=[math.ceil(k_vishkovi[i]*tabela['Izlaz'][i]) for i in range(n)]
tabela['Vishak']=vishkovi

Rachunam ukupan vishak.

In [0]:
vishak=sum(tabela['Vishak'])

Odredjujem kojim apotekam rasporedjujem vishkove.

In [0]:
indeksiApotekaSaManjkom_init=[i for i in range(n) if tabela['Vishak'][i]==0]
k_opt=(sum(tabela['Stanje'][indeksiApotekaSaManjkom_init])+vishak)/sum(tabela['Izlaz'][indeksiApotekaSaManjkom_init])
indeksiApotekaSaManjkom=[i for i in indeksiApotekaSaManjkom_init if tabela['Kolicnik'][i]<k_opt]
while indeksiApotekaSaManjkom_init != indeksiApotekaSaManjkom:
  indeksiApotekaSaManjkom_init=indeksiApotekaSaManjkom
  k_opt=(sum(tabela['Stanje'][indeksiApotekaSaManjkom_init])+vishak)/sum(tabela['Izlaz'][indeksiApotekaSaManjkom_init])
  indeksiApotekaSaManjkom=[i for i in indeksiApotekaSaManjkom_init if tabela['Kolicnik'][i]<k_opt]

Rasporedjujem lekove po apotekama i pravim kolone:
Promena Stanja, Novo Stanje i Novi Kolicnik.

In [0]:
def PromenaStanja(i):
  if i in indeksiApotekaSaManjkom:
    return round(k_opt*tabela['Izlaz'][i]-tabela['Stanje'][i],0)
  else:
    return -(tabela['Vishak'][i])

tabela['Promena Stanja']=[PromenaStanja(i) for i in range(n)]
tabela['Novo Stanje']=tabela['Stanje']+tabela['Promena Stanja']
tabela['Novi Kolicnik']=tabela["Novo Stanje"]/tabela["Izlaz"]

Cekiram da li ima ne slaganja zbog zaokruzhivanja i ako ima resavam ne slaganje.

In [10]:
def resiNeslaganje(e):
  if e>0:
    podTabela=(tabela[tabela['Promena Stanja']>0]).sort_values(by=['Novi Kolicnik'],ascending=False)
    redovi=[i for i in podTabela.index]
    for i in range(e):
      tabela['Promena Stanja'][redovi[i]]-=1
  elif e<0:
    podTabela=(tabela[tabela['Promena Stanja']>0]).sort_values(by=['Novi Kolicnik'])
    redovi=[i for i in podTabela.index]
    for i in range(abs(e)):
      tabela['Promena Stanja'][redovi[i]]+=1

neslaganje=sum(tabela['Promena Stanja'])
if neslaganje != 0:
  resiNeslaganje(int(neslaganje))
tabela['Novo Stanje']=tabela['Stanje']+tabela['Promena Stanja']
tabela['Novi Kolicnik']=tabela["Novo Stanje"]/tabela["Izlaz"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Chekiram da li je sve ok!

In [11]:
sum(tabela['Promena Stanja'])==0

True

Stampam tabelu :)

In [12]:
tabela

,Apoteka,Izlaz,Stanje,Kolicnik,Vishak,Promena Stanja,Novo Stanje,Novi Kolicnik
0,28,15.000,7.0,0.466667,0,11.0,18.0,1.200000
1,2,24.000,10.0,0.416667,0,19.0,29.0,1.208333
2,31,12.000,53.0,4.416667,30,-30.0,23.0,1.916667
3,5,1.000,1.0,1.000000,0,0.0,1.0,1.000000
4,9,7.000,3.0,0.428571,0,6.0,9.0,1.285714
5,21,9.000,3.0,0.333333,0,8.0,11.0,1.222222
6,24,8.000,3.0,0.375000,0,7.0,10.0,1.250000
7,34,10.000,11.0,1.100000,0,1.0,12.0,1.200000
8,39,9.000,11.0,1.222222,0,0.0,11.0,1.222222
9,40,2.000,3.0,1.500000,0,0.0,3.0,1.500000
